In [3]:
import pandas as pd

In [5]:
concentration = pd.read_csv('../../data/2020/20_concentrations.csv')
labels = pd.read_csv('../../data/2020/20_yellow_labels.csv')
pit = pd.read_csv('../../data/2020/20_time_since_pit.csv')
weather = pd.read_csv('../../data/weather_data/2020_weather.csv', sep=';')
speed = pd.read_csv('../../data/2020/20_avg_speed.csv')

In [6]:
max_concentration = pd.DataFrame(columns=['Time', 'Max Concentration'])
temp_max = 0
for index, row in concentration.iterrows():
    sector = row[1]
    if sector == 1:
        temp_max = 0
    if row[2] > temp_max:
        temp_max = row[2]
    if sector == 13:
        max_concentration.loc[index] = [row[0], temp_max]

In [7]:
start_time = weather['TIME_UTC_SECONDS'].iloc[0]

with open('../../data/2020/20_weather_mod.csv', 'w') as f:
    f.write('Time,Air Temp,Track Temp,Wind Speed\n')
    for index, row in weather.iterrows():
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time)//20 * 20, row[2],row[3], row[6]))
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time+20)//20 * 20,row[2],row[3], row[6]))
            f.write("%s,%s,%s,%s\n"%((row[0]-start_time+40)//20 * 20,row[2],row[3], row[6]))
weather_mod = pd.read_csv('../../data/2020/20_weather_mod.csv')
# weather_mod[weather_mod['Time'] == 16940]
weather_mod

Time  Air Temp  Track Temp  Wind Speed
0         0  68.39996    87.00008         1.9
1        20  68.39996    87.00008         1.9
2        40  68.39996    87.00008         1.9
3        60  68.30006    87.99998         1.9
4        80  68.30006    87.99998         1.9
...     ...       ...         ...         ...
4309  86300  64.00004    68.00000         1.3
4310  86320  64.00004    68.00000         1.3
4311  86340  64.00004    68.00000         1.3
4312  86360  64.00004    68.00000         1.3
4313  86380  64.00004    68.00000         1.3

[4314 rows x 4 columns]

In [8]:
agg = max_concentration.merge(pit, how='inner', on='Time')
agg = agg.merge(weather_mod, how='inner', on='Time')
# agg = agg.merge(speed, how='inner', on='Time')
agg = agg.merge(labels, how='inner', on='Time')
agg

Time  Max Concentration     Last Pit  Air Temp  Track Temp  \
0        20.0          21.697795     0.333333  68.39996    87.00008   
1        40.0          15.306122     0.666667  68.39996    87.00008   
2        60.0          15.456989     1.000000  68.30006    87.99998   
3        80.0          15.406162     1.333333  68.30006    87.99998   
4       100.0          15.498652     1.666667  68.30006    87.99998   
...       ...                ...          ...       ...         ...   
4308  86300.0           2.976190  1072.787517  64.00004    68.00000   
4309  86320.0           2.826983  1073.120850  64.00004    68.00000   
4310  86340.0           3.826531  1073.454183  64.00004    68.00000   
4311  86360.0           2.803083  1073.787517  64.00004    68.00000   
4312  86380.0           4.118051  1074.120850  64.00004    68.00000   

      Wind Speed  Yellow  
0            1.9       0  
1            1.9       0  
2            1.9       0  
3            1.9       0  
4            1.9       0  
...          ...     ...  
4308         1.3       0  
4309         1.3       0  
4310         1.3       0  
4311         1.3       0  
4312         1.3       0  

[4313 rows x 7 columns]

In [9]:
agg.to_csv('../../data/2020/20_agg_data.csv', index=False)

In [1]:
times = [30060, 41580]
times = [t//60*60 for t in times]
MAXTIME = 43200
with open('../../data/2020/20_flag_times.csv', 'w') as f:
    f.write('Time,Flag\n')
    for i in range(0,MAXTIME,60):
        if i+120 in times:
            f.write('%s,1\n'%(i))
            f.write('%s,1\n'%(i+60))
            f.write('%s,1\n'%(i+120))
            i = i + 600
        else:
            f.write('%s,0\n'%(i))

In [4]:
labels = pd.read_csv('../../data/2020/20_flag_times.csv')
risk = pd.read_csv('../../data/2020/20_risk_score.csv')
sus = pd.read_csv('../../data/2020/20_sus_deriv.csv')
agg = risk.merge(sus, how='inner', on='Time')
agg = agg.merge(labels, how='inner', on='Time')
agg.to_csv('../../data/2020/20_agg_score.csv', index=False)